##  Using HuggingFace models on AWS Sagemaker for `text-to-sql` operations

### Environment setup (using conda)

Using Conda facilitates dependency management by creating isolated environments. Each environment can have distinct packages, preventing version conflicts. 

Once a you open a new terminal, the base environment of this notebook can be activated and pip can be used within it, combining pip's extensive package availability with Conda's isolation benefits. 

This approach ensures both flexibility and stability in projects, promoting consistent code execution across different setups.

### Opening the Notebook Terminal

On the top pane of this notebook, you should see an icon that looks like a terminal (a square with an underscore).
Click on this icon. A new terminal tab will be opened.

List the conda environments by running the following on the terminal window [ NOT ON THIS NOTEBOOK ]

> conda env list

### Activate a conda environment

To activate any conda environment, run the following command in the terminal window [ NOT ON THIS NOTEBOOK ]

> conda activate base

<div class="alert alert-block alert-info"> <b>NOTE</b> After installing the dependencies on the CLI, you may get warnings mentioning some depedencies are old / outdated. For the scope of this workshop we are not going to train any model instead we will only use the existing huggingface model, hence these messasges can be ignored since the below code will work regardless of those wranings.</div>

Execute below commands on the terminal window [ NOT ON THIS NOTEBOOK ]

> pip install transformers tensorflow torch torchvision pytorch_lightning sentencepiece

### Using the 🤗/mrm8488/t5-base-finetuned-wikiSQL model

<div class="alert alert-block alert-info"> <b>NOTE</b> Run below cell on the NOTEBOOK by pressing `Crtl + ⏎` OR `⌘ + ⏎`.</div>

In [ ]:
import json

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "mrm8488/t5-base-finetuned-wikiSQL"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=200)

<div class="alert alert-block alert-info"> <b>NOTE</b> Run below cell on the NOTEBOOK by pressing `Crtl + ⏎` OR `⌘ + ⏎`.</div>

In [ ]:

def text_to_SQL(input_text):

    # Prepending english to sql for the incoming prompt text
    output = generator("translate English to SQL: %s </s>" % input_text)

    # Extract the first key from the output
    json_output = json.dumps(output[0])

    # Parsing the output as JSON
    sql_query = json.loads(json_output)

    # Return the generated reseponse from the model
    return sql_query['generated_text']

<div class="alert alert-block alert-info"> <b>NOTE</b> Run below cell on the NOTEBOOK by pressing `Crtl + ⏎` OR `⌘ + ⏎`.</div>

In [ ]:
text_to_SQL("What is the average age of the respondents using a mobile device?")

### Trying out the model with few prompts to convert our text input to SQL output

<div class="alert alert-block alert-info"> <b>NOTE</b> Once the SQL is generated, you may still need to tweak the output for use with your database. <br/><br/>Especially tweaks like changing the table name, adding delimiters to the column name and the values etc..</div>

<div class="alert alert-block alert-warning"> <b>NOTE</b><br/> Ensure before you run any SQL command on the MySQL client of the Cloud9 instance, please understand the table's structure for which you are running DESCRIBE command. <br/> We can get the table's structure by running DESCRIBE `tablename`; <br/><br/>Example <br/>For customers table <br/>mysql> DESCRIBE customers;<br/><br/>For employees table <br/>mysql> DESCRIBE employees;<br/><br/>Do not forget to add the ; at the end of each SQL statement.</div>

### Working with our GenAI model to convert text-to-sql

<div class="alert alert-block alert-warning"><b>NOTE</b> The quality of a model completely depends upon the quality of the dataset used while training. <br/><br/> This first model <a href='https://huggingface.co/mrm8488/t5-base-finetuned-wikiSQL' target='_blank'>mrm8488/t5-base-finetuned-wikiSQL</a> was trained on a very small dataset size of 56.4k observations named the <a href='https://huggingface.co/datasets/wikisql' target='_blank'>wikiSQL dataset</a>.</div>

### 1. Generating with keywords to retrieve the top or bottom records from a dataset

> In SQL, the keywords to retrieve the top or bottom records from a dataset vary across different database management systems (DBMS). <br/><br/>  - Microsoft SQL Server (Transact-SQL) uses `Top` syntax.  <br/> - Oracle doesn't have a TOP keyword. You can use the ROWNUM or, in Oracle 12c and later, the FETCH FIRST clause. <br/> - MySQL / PostgresSQL / SQLlite does not have a TOP keyword. Instead, you use the LIMIT clause.

In [ ]:
text_to_SQL("Display top 10 rows of the dataset") # Consider using LIMIT clause while running generated SQL query on Cloud9

### 2. Generating with conditional statements

> Let's get the AI to generate simple query with the use of `WHERE` clause

In [ ]:
text_to_SQL("Get customerName, city from customers table where phone is '61-7-3844-6555'?")

### 3. Generating statements to check null values in our dataset

> In SQL, In all these databases, you cannot use equality ( = ) or inequality ( < > or != ) operators to check for NULL values. For instance, column_name = NULL will not work as you might expect. Always use IS NULL or IS NOT NULL to check for NULL values.

In [ ]:
text_to_SQL("Get customerName values from customers table where addressLine2 is NULL?") # Consider using IS clause instead of using = when running on Cloud9

### Generic examples for observation [ No need to execute the below output on mysql client ]

In [ ]:
text_to_SQL("How many people have French 'fr-en' as their language?")

In [ ]:
text_to_SQL("Job title contains Engineer or engineer")

In [ ]:
text_to_SQL("Find the email of the person with the following IP Address: 132.207.160.22")

In [ ]:
text_to_SQL("How many people have Mastercard as their Credit Card provider and made a purchase above 50?")

In [ ]:
text_to_SQL("Find the email of the person with the following Credit Card Number: 4664825258997302")

In [ ]:
text_to_SQL("How many people have Credit Card which expires in 2024?")

### With a hang of how Gen AI output needs tweaking for most of the outputs, let's observe outputs for below text queries [ Try tweaking and executing on mysql client ]

<div class="alert alert-block alert-warning">Use below prompts and use the `text_to_SQL` function placeholder to generate SQL statements.</div>

> Always run DESCRIBE `tablename`; so that you can understand the table structure to know the column names.

1. Find the maximum amount on the payments table
2. Find the miminum amount on the payments table
3. Find the average amount on the payments table
4. Find the customerName who has made the highest payment on the pyaments table
5. Find the number of employees who has a job title as 'Sales Rep' from the employees table
6. Get the firstName and lastName of employees who reports to 1056 
7. How many orders have been shipped status

In [ ]:
# text_to_SQL function placeholder
# Copy the prompts from above and paste it here and use this cell to execute

text_to_SQL("")